In [2]:
# RNN

In [ ]:
import pandas as pd
import numpy as np
import torch

# 데이터 불러오기
df_train = pd.read_csv("hw2_dataset2_train.csv")
df_test = pd.read_csv("hw2_dataset2_test.csv")
x_cols = df_train.columns[:151]
y_cols = df_train.columns[151:]

X_train = df_train[x_cols].values  # (800, 151)
Y_train = df_train[y_cols].values  # (800, 150)

# Hyperparameters
input_size = 1
hidden_size = 32
output_size = 1
lr = 0.001
epochs = 500
window_size = 10  # 슬라이딩 윈도우 크기

# 슬라이딩 윈도우 적용 (X → y=X_{t+1})
x_seq_list, y_seq_list = [], []
for seq in X_train:
    for i in range(len(seq) - window_size):
        x_seq = seq[i:i + window_size].reshape(-1, 1)
        y_val = seq[i + window_size]  # 다음 시점 값
        x_seq_list.append(x_seq)
        y_seq_list.append([y_val])

X = torch.tensor(x_seq_list, dtype=torch.float32)  # (batch, window, 1)
Y = torch.tensor(y_seq_list, dtype=torch.float32)  # (batch, 1)

# 파라미터 초기화
W_xh = torch.nn.Parameter(torch.randn(input_size, hidden_size) * 0.01)
W_hh = torch.nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
b_h = torch.nn.Parameter(torch.zeros(hidden_size))
W_hy = torch.nn.Parameter(torch.randn(hidden_size, output_size) * 0.01)
b_y = torch.nn.Parameter(torch.zeros(output_size))

params = [W_xh, W_hh, b_h, W_hy, b_y]
optimizer = torch.optim.Adam(params, lr=lr)
loss_fn = torch.nn.MSELoss()

# 학습
for epoch in range(epochs):
    total_loss = 0
    for i in range(X.shape[0]):
        x_seq = X[i]  # (window_size, 1)
        y_true = Y[i]  # (1,)
        h = torch.zeros(hidden_size)

        for t in range(window_size):
            x_t = x_seq[t]
            h = torch.tanh(x_t @ W_xh + h @ W_hh + b_h)

        y_pred = h @ W_hy + b_y
        loss = loss_fn(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss / len(X):.6f}")


C:\Users\epoin\AppData\Local\Temp\ipykernel_16980\920942762.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X = torch.tensor(x_seq_list, dtype=torch.float32)  # (batch, window, 1)


KeyboardInterrupt: 